# 6.6 RetrievalQA

마침내 우리의 document를 이용해 LLM에게 질문을 해볼 시간이 됐다. 지금은 off-the-shelf chain들을 사용할 것이다. "off-the-shelf chain"은 바로 사용 가능한 체인 정도로 보면 된다. 즉, 복잡한 설정 없이 바로 가져다 쓸 수 있는 미리 준비된 체인이라고 생각하면 된다. Langchain에서 미리 만들어 놓은 완성된 체인을 가져다 쓸 수 있다. 편하게 사용 가능하지만 우리들의 입맛에 맞게 커스텀 하는게 까다롭다는 단점이 있다.

이것들을 조금 길게 살펴보고 나서, 우리만의 chain을 LangChain Expression Language으로 만들것이다.

우리가 첫번째로 사용해 볼 것은 Stuff라고 한다. 우리가 찾은 document들로 prompt를 stuff(채우기)하는데 사용한다는 뜻이다.<br>
먼저 질문을 한다. 그럼 그 질문을 사용해서 document를 search(검색)하게 된다. 그 다음 찾은 document들을 Prompt에 입력해서 model에게 전달한다. 그리고 model은 입력된 질문과 documents를 토대로 우리에게 답변해줄 것이다.<br>
여기에는 우리를 위해 이미 만들어져있는(off-the-shelf) 체인이 있다. 그러나 공식 문서를 보면 LCEL로 어떻게 만드는지만 언급되어 있고, off-the-shelf체인은 언급이 안되어 있다. 그러나 우리는 이 off-the-shelf체인을 써 볼 것이다. 그러고나서 우리의 chain을 만들어 보자.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("../files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./.cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
     embeddings,
     cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # 여기서 stuff를 명시해준다.
    retriever=vectorstore.as_retriever(),
    # RetrievalQA chain은 "retriever"도 입력값으로 요구한다.
    # 문서에 따르면, retriever는 class의 interface라고 한다.
    # document를 많은 장소로부터 Retrieve(선별하여 가져오기) 할 수 있다. vector store말고 다른데서도 말이다.
    # 그러니까 retriever를 이용하면 document들을 database에서 검색할 수도 있고,
    # cloude나 vector store에서 찾아올 수도 있다.
    # 정리하면 retriever는 document를 검색해서 찾아오는(retrieve)기능을 가지고 있다.
)

chain.run("What is the name of Oldwyn’s mechanical doll?")


"The name of Oldwyn's mechanical doll is Vesper."

지금 하는것들은 너무 마법같아 보인다. RetrievalQA chain에 대해 더 알고싶다면 공식 문서를 찾아보면 된다. 앞서 말했듯이 우리는 작동을 확인한 후에 우리 손으로 직접 구현해볼 것이다.

다시 한버 stuff에 대해 설명하자면, stuff는 기본적으로 모든 document를 prompt에 채워(stuff)넣는 것을 의미한다. 만약 'foo는 무엇인가요?'라는 질문을 한다면, 'foo'에 관한 document를 받고 모든 document들을 prompt에 삽입해주는 것이다. 그래서 prompt가 꽤나 큰데, 우리가 찾은 모든 document가 입력되었기 때문이다. 그렇게 찾은 documents와 question을 model에게 전달하면서 작동하는 것이다.

Refine이라는 chain도 있다.<br>
여기에서도 만약에 'foo는 무엇입니까?'같은 질문을 하면 foo와 관련된 documents를 얻는것을 같다. 그리고 나서 각각의 document를 읽으면서, 질문에 대한 답변 생성을 시도한다. 즉 'foo는 무엇입니까'와 개별 document를 통해서 답변 생성을 시도한다는 뜻이다. 그 작업을 반복하면서 만나는 모든 document를 통해 question을 개선시키게 된다. 그 작업을 바로 Refine(정제, 가다듬기)라고 한다.<br>
처음에는 질이 좋지않은 답변으로 시작하고, 점차 갈수록 답변을 개선시키는 방식이다. 당연이 이 chain은 비싸다. chain내부에거 각 document에 대한 답변을 생성하는 작업을 또 수행해야 되기 때문이다.

Map Reduce라는 chain도 있다.<br>
이건 document들을 입력받아서, 개별적으로 질문에 관련된 문장을 추출하는 작업을 수행한다. query(질문)를 입력하면, documents들을 얻어서 각각에 대한 추출 작업을 하게 된다. 그리고 각각의 추출본을 LLM에게 전달해준다. 그러니까 굉장히 크고 많은 연산(operation)들이 수행되는 것이다.

Map Re-rank라는 chain도 있다.<br>
여기서도 'foo는 무엇입니까?'라는 질문을 하면 관련된 document들을 받는다. 그러면 각 document를 통해 답변을 생성하고, 각 답변에 점수를 매긴다. 그리고 최종적으로는 가장 높은 점수를 획득한 답변과 그 점수를 함께 반환한다.

이러한 RetrievalQA를 사용할때의 장점중 하나가, chain_type을 손쉽게 바꿀 수 있다는 것이다. 각각의 chain_type은 문서에서 그 구현방법을 볼 수 있다.